In [ ]:
# RUN THIS COMMAND ONLY IF YOU USE GOOGLE COLAB.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# RUN THIS COMMAND ONLY IF YOU USE GOOGLE COLAB.
%cd drive/MyDrive/TechLabs/02_Data\ Manipulation/Part\ C\ -\ Pandas

/content/drive/MyDrive/TechLabs/02_Data Manipulation/Part C - Pandas


In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

In [ ]:
# Load data and pivot it
data = pd.read_csv('mapgraphrecycle.csv')
alldata = pd.pivot_table(data, values="recycling_rate_municipal_waste_percent", index=["country_name"], columns="year", aggfunc=np.mean)

In [ ]:
alldata.head()

year,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
country_name,,,,,,,,,,,,,,,,,,,,,
Albania,18.40,18.40,18.40,18.40,18.40,18.40,18.40,18.4,18.4,18.4,...,18.4,18.4,18.4,18.1,18.70,18.40,18.40,18.40,18.40,18.41
Austria,63.40,64.30,62.50,64.20,57.40,58.30,59.20,60.2,63.2,61.9,...,57.8,57.7,58.2,62.3,59.66,57.19,56.98,56.76,56.55,56.33
Belgium,49.70,50.40,51.50,52.90,53.50,55.60,56.10,56.8,52.7,53.9,...,53.9,54.4,54.7,51.4,53.30,54.20,54.26,54.32,54.38,54.44
Bulgaria,15.50,16.10,16.30,16.50,17.20,18.30,19.10,20.6,19.4,19.9,...,34.6,31.5,34.6,65.5,25.41,39.89,41.15,42.41,43.67,44.93
Croatia,16.38,16.38,16.38,16.38,16.38,16.38,16.38,3.2,2.7,2.4,...,23.6,25.3,30.2,29.5,31.40,24.74,25.47,26.19,26.92,27.65


In [ ]:
alldata=alldata.drop('European Union - 27 countries (from 2020)')

In [ ]:
# Create dataframes for each year
years = [2020, 2021, 2022, 2023, 2024, 2025, 2026]
dataframes = {}
for year in years:
    df = alldata[[year]].reset_index()
    df.columns = ['country_name', str(year)]
    df = df.set_index('country_name')
    dataframes[year] = df

In [ ]:
# Create choropleth map with dropdown for year selection
eu_map = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
eu_map = eu_map[eu_map.continent == 'Europe']

In [ ]:
@interact(year=widgets.Dropdown(options=years, value=2020, description='Year:'))
def create_choropleth_map(year):
    data = dataframes[year]
    eu_data = eu_map.merge(data, left_on='name', right_index=True)

    fig, ax = plt.subplots(figsize=(12, 8), dpi=200)
    eu_data.plot(column=str(year), cmap='YlGn', ax=ax, legend=True, edgecolor='white', 
                 legend_kwds={'label': f'Recycling rate of municipal waste in % ({year})'})

    ax.set_axis_off()
    ax.set_xlim([-20, 40])
    ax.set_ylim([30, 70])

    plt.show()


interactive(children=(Dropdown(description='Year:', options=(2020, 2021, 2022, 2023, 2024, 2025, 2026), value=…